In [8]:
import torch
import torch.nn as nn
from tqdm import tqdm
import sys
import numpy as np
import os
sys.path.append(os.path.abspath('../data'))
from ListOpsDataset import ListOpsDataset
from torch.utils.data import DataLoader


In [9]:

# Define the data directory and file paths
data_dir = '../data/output_dir/'
train_file = os.path.join(data_dir, 'basic_train.tsv')
val_file = os.path.join(data_dir, 'basic_val.tsv')
test_file = os.path.join(data_dir, 'basic_test.tsv')

def load_listops_data(file_path, max_rows=None):
    """
    Load ListOps data from TSV file.
    
    Args:
        file_path: Path to the TSV file
        max_rows: Maximum number of rows to load (for testing)
    
    Returns:
        sources: Array of source expressions
        targets: Array of target values (0-9)
    """
    sources = []
    targets = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        next(f)  # Skip header (Source, Target)
        for i, line in enumerate(f):
            if max_rows and i >= max_rows:
                break
            if not line.strip():  # Skip empty lines
                continue
            parts = line.strip().split('\t')
            if len(parts) != 2:
                continue  # Skip lines that don't have exactly two columns
            source, target = parts
            sources.append(source)
            targets.append(int(target))  # Target is always 0-9
    
    # Convert to numpy arrays
    source_array = np.array(sources, dtype=object)  # Keep expressions as strings
    target_array = np.array(targets, dtype=np.int32)  # Targets are integers
    
    return source_array, target_array

try:
    # Load training data
    print("Loading training data...")
    X_train, y_train = load_listops_data(train_file)
    
    # Load validation data
    print("Loading validation data...")
    X_val, y_val = load_listops_data(val_file)
    
    # Load test data
    print("Loading test data...")
    X_test, y_test = load_listops_data(test_file)
    
    # Print dataset statistics
    print("\nDataset sizes:")
    print(f"Training: {len(X_train)} examples")
    print(f"Validation: {len(X_val)} examples")
    print(f"Test: {len(X_test)} examples")
      
except Exception as e:
    print(f"Error occurred: {type(e).__name__}: {str(e)}")

Loading training data...
Loading validation data...
Loading test data...

Dataset sizes:
Training: 96000 examples
Validation: 2000 examples
Test: 2000 examples


In [10]:

# Create datasets
train_dataset = ListOpsDataset(X_train, y_train)
val_dataset = ListOpsDataset(X_val, y_val)
test_dataset = ListOpsDataset(X_test, y_test)

# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Verify the data
print("Dataset sizes:")
print(f"Train: {len(train_dataset)}")
print(f"Val: {len(val_dataset)}")
print(f"Test: {len(test_dataset)}")

# Check first batch
batch = next(iter(train_loader))
print("\nFirst batch shape:")
print(f"Input IDs: {batch['input_ids'].shape}")
print(f"Targets: {batch['target'].shape}")

Dataset sizes:
Train: 96000
Val: 2000
Test: 2000

First batch shape:
Input IDs: torch.Size([32, 2000])
Targets: torch.Size([32])


In [11]:

# Training function
def train_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        targets = batch['target'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Calculate accuracy
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    return total_loss / len(train_loader), 100. * correct / total

# Validation function
def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            targets = batch['target'].to(device)
            
            outputs = model(input_ids)
            loss = criterion(outputs, targets)
            
            total_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    return total_loss / len(val_loader), 100. * correct / total

# Initialize model and training components
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [14]:
class SimpleRNNModel(nn.Module):
    def __init__(self, 
                 vocab_size=18,      # Size of vocabulary
                 embedding_dim=32,    # Small embedding dimension
                 hidden_dim=64,      # Hidden state dimension
                 num_layers=1,       # Single RNN layer
                 dropout=0.1):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_dim, 10)  #   (0-9)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # x shape: (batch_size, sequence_length)
        x = self.embedding(x)        # (batch_size, sequence_length, embedding_dim)
        x = self.dropout(x)
        
        # RNN forward pass
        output, hidden = self.rnn(x)  # output: (batch_size, sequence_length, hidden_dim)
        
        # Use the last hidden state for classification
        x = hidden[-1]              # (batch_size, hidden_dim)
        x = self.fc(x)              # (batch_size, 10)
        
        return x

# Initialize model and training components
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleRNNModel().to(device)
criterion = nn.CrossEntropyLoss()
#focal loss
criterion = nn.FocalLoss(gamma=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(f"Using device: {device}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")

# Training loop
num_epochs = 5
best_val_acc = 0

print("\nStarting training...")
for epoch in range(num_epochs):
    # Train
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    for batch in pbar:
        input_ids = batch['input_ids'].to(device)
        targets = batch['target'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        # Update progress bar
        pbar.set_postfix({'loss': f'{train_loss/total:.4f}', 
                         'acc': f'{100.*correct/total:.2f}%'})
    
    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    print(f'Epoch: {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss/len(train_loader):.4f} | Train Acc: {100.*correct/total:.2f}%')
    print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%')
    
   
    print('-' * 50)

# Test
test_loss, test_acc = validate(model, test_loader, criterion, device)
print(f'\nTest Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%')

Using device: cpu
Total parameters: 7498

Starting training...


Epoch 1/5: 100%|██████████| 3000/3000 [05:22<00:00,  9.30it/s, loss=0.0705, acc=17.01%]


Epoch: 1/5
Train Loss: 2.2545 | Train Acc: 17.01%
Val Loss: 2.2598 | Val Acc: 17.15%
--------------------------------------------------


Epoch 2/5: 100%|██████████| 3000/3000 [05:20<00:00,  9.35it/s, loss=0.0704, acc=16.82%]


Epoch: 2/5
Train Loss: 2.2515 | Train Acc: 16.82%
Val Loss: 2.2575 | Val Acc: 17.15%
--------------------------------------------------


...

Epoch: 3/5
Train Loss: 2.2536 | Train Acc: 16.84%
Val Loss: 2.2672 | Val Acc: 15.95%
--------------------------------------------------


Epoch 4/5: 100%|██████████| 3000/3000 [05:28<00:00,  9.12it/s, loss=0.0705, acc=16.84%]


Epoch: 4/5
Train Loss: 2.2559 | Train Acc: 16.84%
Val Loss: 2.2778 | Val Acc: 15.95%
--------------------------------------------------


Epoch 5/5: 100%|██████████| 3000/3000 [05:35<00:00,  8.93it/s, loss=0.0705, acc=16.75%]


Epoch: 5/5
Train Loss: 2.2553 | Train Acc: 16.75%
Val Loss: 2.2637 | Val Acc: 17.15%
--------------------------------------------------

Test Loss: 2.2508 | Test Acc: 17.80%


In [16]:
# First, add this class definition for Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(reduction='none')
        
    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * ce_loss
        return focal_loss.mean()

# Then modify the criterion initialization
criterion = FocalLoss(gamma=2)

Focal loss

In [17]:

# Initialize model and training components
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleRNNModel().to(device)
#focal loss
criterion = FocalLoss(gamma=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(f"Using device: {device}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")

# Training loop
num_epochs = 3
best_val_acc = 0

print("\nStarting training...")
for epoch in range(num_epochs):
    # Train
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    for batch in pbar:
        input_ids = batch['input_ids'].to(device)
        targets = batch['target'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        # Update progress bar
        pbar.set_postfix({'loss': f'{train_loss/total:.4f}', 
                         'acc': f'{100.*correct/total:.2f}%'})
    
    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    print(f'Epoch: {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss/len(train_loader):.4f} | Train Acc: {100.*correct/total:.2f}%')
    print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%')
    
   
    print('-' * 50)

# Test
test_loss, test_acc = validate(model, test_loader, criterion, device)
print(f'\nTest Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%')

Using device: cpu
Total parameters: 7498

Starting training...


Epoch 1/3: 100%|██████████| 3000/3000 [05:36<00:00,  8.90it/s, loss=0.0565, acc=16.75%]


Epoch: 1/3
Train Loss: 1.8067 | Train Acc: 16.75%
Val Loss: 1.8135 | Val Acc: 16.00%
--------------------------------------------------


Epoch 2/3: 100%|██████████| 3000/3000 [05:38<00:00,  8.86it/s, loss=0.0565, acc=16.80%]


Epoch: 2/3
Train Loss: 1.8065 | Train Acc: 16.80%
Val Loss: 1.8265 | Val Acc: 15.95%
--------------------------------------------------


Epoch 3/3: 100%|██████████| 3000/3000 [05:37<00:00,  8.89it/s, loss=0.0565, acc=16.96%]


Epoch: 3/3
Train Loss: 1.8085 | Train Acc: 16.96%
Val Loss: 1.8115 | Val Acc: 17.15%
--------------------------------------------------

Test Loss: 1.7908 | Test Acc: 17.80%
